In [ ]:
# Import required libraries
from utils_clean import *
import cv2
import numpy as np

# Initialize variables
w, h = 360, 240  # Image width and height
center = w // 2  # Center of the image
pid_yaw = [0.7, 0.5, 0]  # PID gains for yaw
pid_fb = [0.7, 0.5, 0]  # PID gains for forward/backward
pid_ud = [0.7, 0.5, 0]  # PID gains for up/down
pError_yaw, pError_fb, pError_ud = 0, 0, 0  # Initialize previous errors
facearea = 0  # Current face area
desiredfacearea = 5000  # Target face area
startCounter = 0  # Takeoff flag

# Initialize drone
myDrone = initializeTello()

# Main loop
while True:

    # Drone takeoff if it hasn't already
    if startCounter == 0:
        myDrone.takeoff()
        startCounter = 1

    # Get video frame and face information
    img = telloGetFrame(myDrone, w, h)
    img, info = findFace(img)
    facearea = findArea(img)

    # Handle keyboard inputs for modifying desired face area and center
    if cv2.waitKey(10) & 0xFF == ord('w'):
        pError_fb = 0
        desiredfacearea += 100
        print('desiredfacearea +10')
    if cv2.waitKey(10) & 0xFF == ord('s'):
        pError_fb = 0
        desiredfacearea -= 100
        print('desiredfacearea -10')
    if cv2.waitKey(10) & 0xFF == ord('a'):
        pError_yaw = 0
        center += 10
        print('center +10')
    if cv2.waitKey(10) & 0xFF == ord('d'):
        pError_yaw = 0
        center -= 10
        print('center -10')

    # Constrain desired face area and center within limits
    desiredfacearea = int(np.clip(desiredfacearea, 3000, 9000))
    center = int(np.clip(center, 0, 360))

    # Update drone's position to track face
    pError_yaw, pError_fb, pError_ud = trackFace(
        myDrone, info, w, pid_yaw, pid_fb, pError_yaw, pError_fb, facearea, h, 
        pError_ud, pid_ud, center, desiredfacearea
    )

    # Display video feed
    cv2.imshow('Image', img)

    # Exit condition
    if cv2.waitKey(1) & 0xFF == ord('q'):
        myDrone.land()
        break


| Code Example                                      | Code Explanation                                                                   |
|---------------------------------------------------|------------------------------------------------------------------------------------|
| `from utils_clean import *`                        | Imports all utility functions.                                                     |
| `import cv2, numpy as np`                          | Imports the required libraries for image processing and numerical calculations.    |
| `w, h = 360, 240`                                  | Sets the image width and height.                                                   |
| `center = w // 2`                                  | Calculates the center of the image.                                                |
| `pid_yaw = [0.7, 0.5, 0]`                          | Initializes the PID gains for yaw control.                                         |
| `pError_yaw, pError_fb, pError_ud = 0, 0, 0`       | Initializes the previous errors for yaw, forward/backward, and up/down.            |
| `facearea = 0`                                     | Initializes the current face area to zero.                                         |
| `desiredfacearea = 5000`                           | Sets the desired face area for tracking.                                           |
| `startCounter = 0`                                 | Initializes the takeoff flag.                                                      |
| `myDrone = initializeTello()`                      | Calls function to initialize the Tello drone.                                      |
| `while True:`                                      | Main loop for the drone control.                                                   |
| `if startCounter == 0:`                            | Checks if the drone has taken off; if not, it takes off.                           |
| `img = telloGetFrame(myDrone, w, h)`               | Fetches the current frame from the drone's camera.                                 |
| `img, info = findFace(img)`                        | Detects faces in the frame and returns the frame and face information.             |
| `facearea = findArea(img)`                         | Gets the area of the largest face detected.                                        |
| `if cv2.waitKey(10) & 0xFF == ord('w'):`           | Checks for 'w' key press to increase the desired face area.                        |
| `desiredfacearea = int(np.clip(desiredfacearea, 3000, 9000))` | Clips the desired face area within limits.            |
| `pError_yaw, pError_fb, pError_ud = trackFace(...)`| Calls the `trackFace` function to adjust the drone's position based on the face area and its position. |
| `cv2.imshow('Image', img)`                         | Displays the video feed.                                                           |
| `if cv2.waitKey(1) & 0xFF == ord('q'):`            | Checks for 'q' key press to land the drone and break the loop.                     |

